# The AI Patent Analyst: From Unstructured PDFs to a Queryable Knowledge Graph

## 1. High-Level Summary

This project demonstrates an end-to-end AI pipeline built entirely within Google BigQuery that transforms unstructured patent PDFs into a structured, queryable Knowledge Graph. The final prototype acts as a sophisticated IP analysis tool, enabling deep architectural analysis, component-level semantic search, all using BigQuery AI functions.

This solution directly addresses the challenge of making sense of messy, mixed-format data that is often overlooked, turning it into a powerful, interactive analytical asset.

## 2. The Workflow: A Multi-Stage AI Pipeline

Our solution follows a three-stage process, showcasing a powerful combination of BigQuery's multimodal, generative, and vector search capabilities.

### Stage 1: Multimodal Data Processing (🖼️ Pioneer)
We use **Object Tables** to directly read and process raw PDFs from Cloud Storage. The Gemini model is then used with `ML.GENERATE_TEXT` to analyze the both the text and the technical diagrams within the PDFs.

### Stage 2: Generative Knowledge Graph Extraction (🧠 Architect)
The consolidated patent text is fed into the `AI.GENERATE_TABLE` function. A custom prompt instructs the AI to act as an expert analyst, extracting a structured table of high-level insights (`invention_domain`, `problem_solved`) and a detailed, nested graph of all technical components, their functions, and their interconnections.

### Stage 3: Component-Level Semantic Search (🕵️‍♀️ Detective)
To enable deep discovery, we build a novel search engine that understands context. We use `ML.GENERATE_EMBEDDING` to create two separate vectors:
1.  One for the patent's high-level context (title, abstract)
2.  Another for each component's specific function

These vectors are mathematically averaged into a single, final vector for each component via BigQuery's UDF (User-Defined Functions).

Finally, `VECTOR_SEARCH` is used on these combined vectors, creating a powerful search that returns highly relevant, context-aware results.

## 3. Key Features & Analytical Capabilities

The final `patent_knowledge_graph` table is not just a dataset; it's an interactive analysis engine that can answer questions which are more difficult and time consuming with the original text:

*   **Deep Architectural Analysis:** Use standard SQL with `UNNEST` and `GROUP BY` to discover the most common design patterns and component connections across hundreds of inventions.

*   **Component Search:** Go beyond patent-level search to find specific, functionally similar technical parts across different domains (e.g., "find a mechanism for encrypting data").


*   **Quantitative Portfolio Analysis:** Compare patent applicants by the complexity (average component count) and breadth (number of domains) of their innovations.


## 4. Dataset Overview
- **403 PDFs** (197 English, others in FR/DE) at `gs://gcs-public-data--labeled-patents/*.pdf`.
- **Tables**: `extracted_data` (metadata), `invention_types` (labels), `figures` (91 diagram coordinates).
- **Source**: [Labeled Patents](https://console.cloud.google.com/marketplace/product/global-patents/labeled-patents?inv=1&invt=Ab5j9A&project=bq-ai-patent-analyst&supportedpurview=organizationId,folder,project) (1TB/mo free tier).

## 5. Code
*   **Notebook & Repository:** [https://github.com/veyselserifoglu/bq-ai-patent-analyst/blob/main/notebooks/bigquery-ai-the-patent-analyst-project.ipynb](https://github.com/veyselserifoglu/bq-ai-patent-analyst/blob/main/notebooks/bigquery-ai-the-patent-analyst-project.ipynb)

## 6. Architecture Pipeline

In [30]:
# Display Architecture pipeline

HTML(f'''
<div style="text-align: center; padding: 15px;">
    <a href="https://github.com/veyselserifoglu/bq-ai-patent-analyst/blob/main/doc/Patent%20Analysis%20Pipeline%20Architecture%20-%20PNG.png?raw=true" 
       target="_blank" 
       style="cursor: pointer; display: inline-block; text-decoration: none;">
        <div style="position: relative; display: inline-block;">
            <img src="https://github.com/veyselserifoglu/bq-ai-patent-analyst/blob/main/doc/Patent%20Analysis%20Pipeline%20Architecture%20-%20PNG.png?raw=true" 
                 width="300" 
                 height="200"
                 style="border: 2px solid #e0e0e0; border-radius: 8px; transition: all 0.3s ease; box-shadow: 0 4px 8px rgba(0,0,0,0.1);"
                 onmouseover="this.style.borderColor='#4285F4'; this.style.boxShadow='0 6px 12px rgba(66, 133, 244, 0.3)'"
                 onmouseout="this.style.borderColor='#e0e0e0'; this.style.boxShadow='0 4px 8px rgba(0,0,0,0.1)'">
            <div style="position: absolute; top: 8px; right: 8px; background: rgba(255,255,255,0.9); border-radius: 50%; width: 24px; height: 24px; display: flex; align-items: center; justify-content: center; font-size: 14px;">
                ↗
            </div>
        </div>
    </a>
    <p style="margin-top: 12px; color: #5f6368; font-size: 13px; font-style: italic;">Click to explore the full architecture</p>
</div>
''')

In [ ]:
# For visualization purposes
%pip install -q pyvis
%pip install -q plotly

In [ ]:
# BigQuery
import os
from google.cloud import bigquery
from kaggle_secrets import UserSecretsClient
import pandas as pd
from pyvis.network import Network
import plotly.express as px
from google.cloud import bigquery
from IPython.display import Image, display, HTML, IFrame

# Google Cloud Project Setup

This guide outlines the one-time setup required in Google Cloud and Kaggle to enable the analysis.

---

### 1. Google Cloud Project Configuration

First, configure your Google Cloud project.

1.  **Select or Create a Project**
    * Ensure you have a Google Cloud project.
    * Copy the **Project ID** (e.g., `my-project-12345`), not the project name.

2.  **Enable Required APIs**
    * In your project, enable the following two APIs:
        * **Vertex AI API**
        * **BigQuery Connection API**

3.  **Create a Service Account for the Notebook**
    * This service account allows the Kaggle notebook to act on your behalf.
    * Navigate to **IAM & Admin** > **Service Accounts**.
    * Click **+ CREATE SERVICE ACCOUNT**.
    * Give it a name (e.g., `kaggle-runner`).
    * Grant it these three roles:
        * `BigQuery Admin`
        * `Vertex AI User`
        * `Service Usage Admin`
    * After creating the account, go to > manage keys > create a new key. A file will be downloaded to your computer.

---

### 2. Kaggle Notebook Configuration

Next, configure this Kaggle notebook to use your project.

1.  **Add Kaggle Secrets**
    * In the notebook editor, go to the **"Add-ons"** menu and select **"Secrets"**.
    * Add two secrets:
        * **`GCP_PROJECT_ID`**: Paste your Google Cloud **Project ID** here.
        * **`GCP_SA_KEY`**: Open the downloaded JSON key file, copy its entire text content, and paste it here.

---

### 3. Final Permission Step (After Running Code)

The first time you run the setup cells in the notebook, a new BigQuery connection will be created. This connection has its own unique service account that needs permission to use AI models.

1.  **Find the Connection Service Account**
    * After running the setup cells, go to **BigQuery** > **External connections** in your Google Cloud project.
    * Click on the connection named `llm-connection`.
    * Copy its **Service Account ID** (it will look like `bqcx-...@...gserviceaccount.com`).

2.  **Grant Permission**
    * Go to the **IAM & Admin** page.
    * Click **+ Grant Access**.
    * Paste the connection's service account ID into the **"New principals"** box.
    * Give it the single role of **`Vertex AI User`**.
    * Click **Save**.

---

With this setup complete, the notebook has secure access to your Google Cloud project and can run all subsequent analysis cells.

In [ ]:
user_secrets = UserSecretsClient()
project_id = user_secrets.get_secret("GCP_PROJECT_ID")
gcp_key_json = user_secrets.get_secret("GCP_SA_KEY")
location = 'US'

In [ ]:
# Write the key to a temporary file in the notebook's environment
key_file_path = 'gcp_key.json'
try:
    with open(key_file_path, 'w') as f:
        f.write(gcp_key_json)
    
    # Remove "> /dev/null 2>&1" to show the output.
    # Authenticate the gcloud tool using the key file
    !gcloud auth activate-service-account --key-file={key_file_path} > /dev/null 2>&1
    
    # Configure the gcloud tool to use your project
    !gcloud config set project {project_id} > /dev/null 2>&1
    
finally:
    # Securely delete the key file immediately after use
    if os.path.exists(key_file_path):
        os.remove(key_file_path)

# Enable the Vertex AI and BigQuery Connection APIs. Run only once Or Enable using the Cloud Interface.
# !gcloud services enable aiplatform.googleapis.com bigqueryconnection.googleapis.com > /dev/null 2>&1

In [ ]:
# This command creates the connection resource. Remove "> /dev/null 2>&1" to show the output.
!bq mk --connection --location={location} --connection_type=CLOUD_RESOURCE llm-connection > /dev/null 2>&1

In [ ]:
# This command shows the details of your connection. Remove "> /dev/null 2>&1" to show the output.
!bq show --connection --location={location} llm-connection > /dev/null 2>&1

# BigQuery Resource Creation

This section runs three commands to create the necessary resources for our analysis inside your BigQuery project.

---

### 1. Create a Dataset in the Correct Region

First, we create a new dataset named `patent_analysis` in our chosen region. This dataset acts as a container for the AI model and the object table we will create next.

### 2. Create a Reference to the AI Model

Next, we create a "shortcut" to Google's `gemini-2.5-flash` model. This command gives us an easy name, `gemini_vision_analyzer`, to use in our analysis queries.

### 3. Create an Object Table for the PDFs

Finally, we create an object table named `patent_documents_object_table`. This is a special "map" that points directly to all the raw PDF files in the public Google Cloud Storage bucket, making them ready for analysis.

---

In [ ]:
client = bigquery.Client(project=project_id, location=location)
client

In [ ]:
# 1. Create the new dataset in the correct location
create_dataset_query = f"""
CREATE SCHEMA IF NOT EXISTS `{project_id}.patent_analysis`
OPTIONS(location = '{location}');
"""

print(f"Creating dataset 'patent_analysis' in {location}...")
job = client.query(create_dataset_query)
try:
    job.result()
    print("✅ Dataset created successfully or already exists.")
except Exception as e:
    print(f"❌ FAILED to create dataset. Error:\n\n{e}")


# 2. Create the AI model reference inside the new dataset
create_model_query = f"""
CREATE OR REPLACE MODEL `{project_id}.patent_analysis.gemini_vision_analyzer`
  REMOTE WITH CONNECTION `{location}.llm-connection`
  OPTIONS (endpoint = 'gemini-2.5-flash');
"""

print("\nCreating the AI model reference...")
job = client.query(create_model_query)
try:
    job.result() # This waits for the job to complete.
    print("✅ SUCCESS: Model 'gemini_vision_analyzer' created successfully.")
except Exception as e:
    print(f"❌ FAILED: The query failed. Please share this full error message:\n\n{e}")

# TODO: replace the gcs files source with a variable
# 3. Create the Object Table
# This query creates the "map" to the PDF files inside the local 'patent_analysis' dataset.
object_table_query = f"""
CREATE OR REPLACE EXTERNAL TABLE `{project_id}.patent_analysis.patent_documents_object_table`
WITH CONNECTION `{location}.llm-connection`
OPTIONS (
    object_metadata = 'SIMPLE',
    uris = ['gs://gcs-public-data--labeled-patents/*.pdf'] 
);
"""

print("Creating the object table...")
job = client.query(object_table_query)
try:
    job.result()
    print("✅ Object table created successfully.")
except Exception as e:
    print(f"❌ FAILED to create the object table. Error:\n\n{e}")

# Data Extraction & Knowledge Graph Creation

## What did we build?
We created the foundational data asset for this entire project: the `patent_knowledge_graph` table. This involved two major extraction steps.

## Why is this important?
- The raw patent PDFs are a classic example of "dark data"; valuable information that is difficult to query and analyze.
- This section details how we transformed that messy, unstructured data into a clean, structured table.
- This unlocks the data, making it possible to perform the deep SQL analysis and advanced semantic search in the later stages of this notebook.

## How did we do it?
The process used a sequence of BigQuery's native AI functions:

1. **Multimodal Analysis**:
   - First, we used `ML.GENERATE_TEXT` to analyze the text and the technical diagrams within each patent's PDF.
   - Generated rich descriptions of the visual components.

2. **Knowledge Graph Extraction**:
   - Next, we fed all the consolidated text into the `AI.GENERATE_TABLE` function.
   - We gave the model a custom prompt, instructing it to act as an expert and extract:
     - A structured, nested table of all technical components.
     - Their functions.
     - Their connections for each patent.

In [ ]:
# 1. Multimodal Analysis - only texts.

prompt_text = """From this patent document, perform the following tasks:

1.  **Extract these fields**: title, inventor, abstract, 
    the **Filed**, the **Date of Patent**, the international classification code, and the applicant.
    
2.  **Translate**: If the original title and abstract are in German or French, translate them into English.

3.  **Identify Language**: Determine the original language of the document.

Return ONLY a valid JSON object with EXACTLY these ten keys: 
"title_en", "inventor", "abstract_en", "filed", "date_of_patent", "class_international", "applicant", and "original_language".

**Formatting Rule**: For any key that has multiple values (like "inventor" or "class_international" or "applicant"), 
combine them into a single string, separated by a comma and a space. For example: "Igor Karp, Lev Stesin".

The "original_language" value must be one of these three strings: 'EN', 'FR', or 'DE'.
If any other field is unavailable, use null as the value.
"""

# The main SQL query.
sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.ai_text_extraction` AS (
  WITH raw_json AS (
      SELECT
        uri,
        ml_generate_text_llm_result AS llm_result
      FROM
        ML.GENERATE_TEXT(
          MODEL `{project_id}.patent_analysis.gemini_vision_analyzer`,
          TABLE `{project_id}.patent_analysis.patent_documents_object_table`,
          STRUCT(
            '''{prompt_text}''' AS prompt,
            2048 AS max_output_tokens,
            0.2 AS temperature,
            TRUE AS flatten_json_output
          )
        )
    ),
    parsed_json AS (
      -- Step 2: Clean and parse the JSON output.
      SELECT
        uri,
        llm_result,
        SAFE.PARSE_JSON(
          REGEXP_REPLACE(llm_result, r'(?s)```json\\n(.*?)\\n```', r'\\1')
        ) AS json_data
      FROM
        raw_json
    )
  SELECT
    uri,
    llm_result,
    
    SAFE.JSON_VALUE(json_data, '$.original_language') AS original_language,
    SAFE.JSON_VALUE(json_data, '$.title_en') AS extracted_title_en,
    SAFE.JSON_VALUE(json_data, '$.inventor') AS extracted_inventor,
    SAFE.JSON_VALUE(json_data, '$.abstract_en') AS extracted_abstract_en,
    SAFE.JSON_VALUE(json_data, '$.filed') AS filed_date,
    SAFE.JSON_VALUE(json_data, '$.date_of_patent') AS official_patent_date,
    SAFE.JSON_VALUE(json_data, '$.class_international') AS class_international,
    SAFE.JSON_VALUE(json_data, '$.applicant') AS applican
    
  FROM
    parsed_json
);
"""

job = client.query(sql_query)

try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [ ]:
# 1. Multimodal Analysis - only technical diagrams.

diagram_prompt_text = """
Describe this technical diagram from a patent document. 
What is its primary function and what key components are labeled?
"""

sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.ai_text_extraction` AS (

  WITH figures_with_object_ref AS (
      SELECT
        fig.*, obj.ref
      FROM
        `bigquery-public-data.labeled_patents.figures` AS fig
      JOIN
        `{project_id}.patent_analysis.patent_documents_object_table` AS obj
      ON
        fig.gcs_path = obj.uri
    ),
    
    generated_descriptions AS (
      SELECT
        gcs_path,
        ml_generate_text_llm_result AS diagram_description
      FROM
        ML.GENERATE_TEXT(
          MODEL `{project_id}.patent_analysis.gemini_vision_analyzer`,
          (
            SELECT
              gcs_path,
              [
                JSON_OBJECT('uri', ref.uri, 'bounding_poly', [
                  STRUCT(x_relative_min AS x, y_relative_min AS y),
                  STRUCT(x_relative_max AS x, y_relative_min AS y),
                  STRUCT(x_relative_max AS x, y_relative_max AS y),
                  STRUCT(x_relative_min AS x, y_relative_max AS y)
                ])
              ] AS contents,
              '''{diagram_prompt_text}''' AS prompt
            FROM
              figures_with_object_ref
          ),
          STRUCT(
            4096 AS max_output_tokens,
            0.2 AS temperature,
            TRUE AS flatten_json_output
          )
        )
    ),

    aggregated_descriptions AS (
      SELECT
        gcs_path,
        ARRAY_AGG(diagram_description IGNORE NULLS) AS diagram_descriptions
      FROM
        generated_descriptions
      GROUP BY
        gcs_path
    )

  SELECT
    T.*,
    S.diagram_descriptions
  FROM
    `{project_id}.patent_analysis.ai_text_extraction` AS T
  LEFT JOIN
    aggregated_descriptions AS S
  ON
    T.uri = S.gcs_path
);
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [ ]:
# 2. Knowledge Graph Extraction.

# Define the schema as a Python variable
schema = """
invention_domain STRING, problem_solved STRING, patent_type STRING, 
components ARRAY<STRUCT<component_name STRING, component_function STRING, connected_to ARRAY<STRING>>>
"""

# The prompt text remains the same
prompt_text = """
From the following patent text, perform these tasks:
1. Determine the high-level technical domain (e.g., 'Telecommunications', 'Medical Devices').
2. Provide a one-sentence summary of the core problem the invention solves.
3. Classify the patent as a 'Method', 'System', 'Apparatus', or a combination.
4. Extract all technical components into a nested list. 
For each component, provide its name, its primary function, and a list of other components it is connected to.

Here is the text:
"""

sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.patent_knowledge_graph` AS (
  SELECT
    t.uri,
    t.invention_domain,
    t.problem_solved,
    t.patent_type,
    t.components
  FROM
    AI.GENERATE_TABLE(
      MODEL `{project_id}.patent_analysis.gemini_vision_analyzer`,
      (
        SELECT
          uri,
          CONCAT(
            '''{prompt_text}''',
            '\\n\\n',
            IFNULL(extracted_title_en, ''),
            '\\n\\n',
            IFNULL(extracted_abstract_en, ''),
            '\\n\\nDiagrams:\\n',
            IFNULL(ARRAY_TO_STRING(diagram_descriptions, '\\n'), '')
          ) AS prompt
        FROM
          `{project_id}.patent_analysis.ai_text_extraction`
        WHERE
          extracted_abstract_en IS NOT NULL
      ),
      STRUCT(
        '''{schema}''' AS output_schema
      )
    ) AS t
);
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

## Visualization

### Strategic Patent Portfolio Analysis

This chart provides a strategic overview of the patent applicants in our dataset. Each bubble represents a single applicant.

---
#### How to Read This Graph:

* **Innovation Breadth (Horizontal Axis):** This measures how diverse an applicant's portfolio is. A position further to the right means the company has filed patents across a wider range of different technical domains.

* **Average Complexity (Vertical Axis):** This is a proxy for how complex an applicant's inventions are. A higher position means their patents, on average, have a greater number of technical **components** as extracted by our AI.

* **Bubble Size:** The size of each bubble corresponds to the **total number of patents** that applicant has in this dataset.

---
#### What It Tells Us:

This visualization helps us quickly identify different types of innovators. For example, a large bubble high up on the chart represents a major player creating complex inventions, while a small bubble far to the right might represent a nimble innovator exploring many different fields.

In [1]:
# This query creates a summary table for each patent applicant.
sql_query = f"""
WITH applicant_data AS (
  SELECT
    T1.applican,
    T2.invention_domain,
    ARRAY_LENGTH(T2.components) AS component_count
  FROM
    `{project_id}.patent_analysis.ai_text_extraction` AS T1
  JOIN
    `{project_id}.patent_analysis.patent_knowledge_graph` AS T2
  ON
    T1.uri = T2.uri
  WHERE
    T1.applican IS NOT NULL AND T2.invention_domain IS NOT NULL
)

SELECT
  applican,
  COUNT(DISTINCT invention_domain) AS innovation_breadth,
  ROUND(AVG(component_count), 2) AS average_complexity,
  COUNT(applican) AS total_patents
FROM
  applicant_data
GROUP BY
  applican
HAVING
  COUNT(applican) > 1
ORDER BY
  total_patents DESC;
"""

df_summary = client.query(sql_query).to_dataframe()

fig = px.scatter(
    df_summary,
    x="innovation_breadth",
    y="average_complexity",
    size="total_patents",          
    color="applican",              
    hover_name="applican",        
    log_x=True,                    
    size_max=60,                   
    title="<b>Strategic Patent Portfolio Analysis: Breadth vs. Complexity</b>",
    labels={
        "innovation_breadth": "Innovation Breadth (Number of Domains)",
        "average_complexity": "Average Invention Complexity (Number of Components)"
    }
)

# Customize the layout for a professional look
fig.update_layout(
    showlegend=False,
    xaxis_title="<b>Innovation Breadth ➡️</b> (More Diverse)",
    yaxis_title="<b>Average Complexity ⬆️</b> (More Complex)"
)

fig.show()

NameError: name 'project_id' is not defined

# Patent Insights with SQL Analysis

## What did we build?

Now that we have transformed the unstructured patent data into a structured Knowledge Graph, we can finally ask it complex questions.

## Why is this important?
- This is the payoff. 
- We will run queries that are impossible to perform on the original text.
- Uncovering quantifiable insights about:
  - Invention complexity
  - Common design patterns across the entire dataset
- Proves the value of the data transformation pipeline.

## What will we find?
We will perform two types of analysis:

1. **Quantitative Analysis**:
   - Compare the average number of components across different technical domains
   - Measure and rank their complexity

2. **Architectural Pattern Mining**:
   - `UNNEST` the component data
   - Finds the most common "building blocks" and design patterns connected to any component we choose.

In [ ]:
# Quantitative Analysis.

sql_query = f"""
SELECT
  invention_domain,
  COUNT(uri) AS number_of_patents,
  ROUND(AVG(ARRAY_LENGTH(components)), 2) AS average_components,
  MIN(ARRAY_LENGTH(components)) AS min_components,
  MAX(ARRAY_LENGTH(components)) AS max_components
FROM
  `{project_id}.patent_analysis.patent_knowledge_graph`
WHERE
  ARRAY_LENGTH(components) > 0
GROUP BY
  invention_domain
ORDER BY
  average_components DESC;
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

df = job.to_dataframe()
df[df['number_of_patents'] >= 4]

In [ ]:
# Architectural Pattern Mining.

searching_topic = "server"

sql_query = f"""
WITH
  patent_components AS (
    SELECT
      t.uri,
      c.component_name,
      c.connected_to
    FROM
      `{project_id}.patent_analysis.patent_knowledge_graph` AS t,
      UNNEST(t.components) AS c
  ),

  component_connections AS (
    SELECT
      pc.uri,
      pc.component_name,
      connected_component
    FROM
      patent_components AS pc,
      UNNEST(pc.connected_to) AS connected_component
  )

SELECT
  connected_component,
  COUNT(connected_component) AS connection_count
FROM
  component_connections
WHERE
  REGEXP_CONTAINS(component_name, r'(?i){searching_topic}')
GROUP BY
  connected_component
ORDER BY
  connection_count DESC
LIMIT 10;
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

df = job.to_dataframe()
df.head(5)

# Patent Search Engine

## What did we build?
A powerful semantic search engine that finds specific technical components based on a natural language description of their function.

## Why is this important?
- Standard search finds keywords. This search finds meaning.
- By combining two different vector embeddings, the engine understands patent's components and the technical context in which it operates.
- This allows an engineer to find a "valve for precise fluid delivery" and get results from relevant medical patents, not car engine patents.

## How did we do it?
The process involves three key stages, all performed within BigQuery:

1. **Dual Embeddings**:
   - We first generate two separate vector embeddings:
     - One for the high-level patent context (title, abstract, domain, diagrams)
     - Another for the specific component's function

2. **Vector Combination**:
   - We then create a custom User-Defined Function (UDF) to mathematically average these two vectors.
   - This creates a single, final vector for each component that is rich with both specific and contextual meaning.

3. **Semantic Search**:
   - Finally, we use the `VECTOR_SEARCH` function to compare a user's query against these combined vectors.
   - Returns the most similar components from the entire dataset.


In [ ]:
# Create a remote connection for our embedding model
sql_query = f"""
CREATE OR REPLACE MODEL `{project_id}.patent_analysis.embedding_model`
  REMOTE WITH CONNECTION `{location}.llm-connection`
  OPTIONS (endpoint = 'gemini-embedding-001');
"""
job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [ ]:
# This query creates a flat table of all components from all patents.
sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.patent_components_flat` AS (
  SELECT
    t.uri,
    t.invention_domain,
    c.component_name,
    c.component_function,
    c.connected_to
  FROM
    `{project_id}.patent_analysis.patent_knowledge_graph` AS t,
    UNNEST(t.components) AS c
  WHERE
    c.component_function IS NOT NULL
    AND c.component_name IS NOT NULL
);
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [ ]:
# This query creates a single context vector for each patent, reading from ai_text_extraction table.
sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.patent_context_embeddings` AS (
  SELECT
    t.uri,
    t.ml_generate_embedding_result AS patent_context_vector
  FROM
    ML.GENERATE_EMBEDDING(
      MODEL `{project_id}.patent_analysis.embedding_model`,
      (
        SELECT
          uri,
          CONCAT(
            'Patent Title: ', IFNULL(extracted_title_en, ''), '\\n\\n',
            'Applicant: ', IFNULL(applican, ''), '\\n\\n',
            'International Classification: ', IFNULL(class_international, ''), '\\n\\n',
            'Abstract: ', IFNULL(extracted_abstract_en, ''), '\\n\\n',
            'Diagram Descriptions: ', IFNULL(ARRAY_TO_STRING(diagram_descriptions, '\\n'), '')
          ) AS content
        FROM
          `{project_id}.patent_analysis.ai_text_extraction`
        WHERE
          extracted_title_en IS NOT NULL
      )
    ) AS t
);
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [ ]:
# This query creates a single specific function vector for each individual component.
sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.component_function_embeddings` AS (
  SELECT
    t.uri,
    t.component_name,
    t.ml_generate_embedding_result AS component_function_vector
  FROM
    ML.GENERATE_EMBEDDING(
      MODEL `{project_id}.patent_analysis.embedding_model`,
      (
        SELECT
          uri,
          component_name,
          CONCAT(
            'A component named "', component_name, '" whose function is to ', component_function
          ) AS content
        FROM
          `{project_id}.patent_analysis.patent_components_flat`
      )
    ) AS t
);
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [ ]:
# This creates a UDF (user defined function), a helper function for averaging the previous two vectors that we created.
sql_query = f"""
CREATE OR REPLACE FUNCTION `{project_id}.patent_analysis.VECTOR_AVG`(vec1 ARRAY<FLOAT64>, vec2 ARRAY<FLOAT64>)
    RETURNS ARRAY<FLOAT64>
    LANGUAGE js AS r'''
      if (!vec1 || !vec2 || vec1.length !== vec2.length) {{
        return null;
      }}
      let avg_vec = [];
      for (let i = 0; i < vec1.length; i++) {{
        avg_vec.push((vec1[i] + vec2[i]) / 2.0);
      }}
      return avg_vec;
    ''';
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [ ]:
# This query joins all data and creates the final, context-aware combined vector.
sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.component_search_index` AS (
  SELECT
    flat.uri,
    flat.component_name,
    flat.component_function,
    `{project_id}.patent_analysis.VECTOR_AVG`(
      ctx.patent_context_vector,
      func.component_function_vector
    ) AS combined_vector
  FROM
    `{project_id}.patent_analysis.patent_components_flat` AS flat
  JOIN
    `{project_id}.patent_analysis.patent_context_embeddings` AS ctx
  ON
    flat.uri = ctx.uri
  JOIN
    `{project_id}.patent_analysis.component_function_embeddings` AS func
  ON
    flat.uri = func.uri AND flat.component_name = func.component_name
);
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [ ]:
searching_prompt = "I want a similar patent to a small network IOT device"

sql_query = f"""
SELECT
  base.uri,
  base.component_name,
  base.component_function,
  distance
FROM
  VECTOR_SEARCH(
    TABLE `{project_id}.patent_analysis.component_search_index`,
    'combined_vector',
    (
      SELECT ml_generate_embedding_result
      FROM ML.GENERATE_EMBEDDING(
        MODEL `{project_id}.patent_analysis.embedding_model`,
        (SELECT '{searching_prompt}' AS content)
      )
    ),
    top_k => 7
  )
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

df = job.to_dataframe()
df.head(5)

In [ ]:
def style_search_results(results_df: pd.DataFrame, search_query: str):
    """
    Takes a DataFrame of search results and returns a styled HTML table
    for clear and compelling presentation.
    """
    if results_df.empty:
        print("⚠️ The search returned no results.")
        return None

    # Create a copy to avoid modifying the original DataFrame
    styled_df = results_df.copy()

    # --- Apply Formatting and Styling ---

    # 1. Make the URI a clickable link
    styled_df['uri'] = styled_df['uri'].apply(lambda x: f'<a href="{x}" target="_blank">{x.split("/")[-1]}</a>')

    # 2. Format the distance column
    styled_df['distance'] = styled_df['distance'].map('{:.4f}'.format)

    # 3. Use the Pandas Styler to add advanced formatting
    styler = styled_df.style \
        .set_caption(f"<h3>Top 5 Component Matches for: '{search_query}'</h3>") \
        .set_properties(**{
            'text-align': 'left',
            'white-space': 'normal', # Allow text wrapping
            'font-size': '14px',
            'font-family': 'Arial, sans-serif'
        }) \
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-size', '16px')]},
            {'selector': 'caption', 'props': [('caption-side', 'top'), ('font-size', '18px')]}
        ]) \
        .hide(axis='index') \
        .background_gradient(subset=['distance'], cmap='RdYlGn_r') # Green for low distance, Red for high

    return styler

styled_table = style_search_results(df, searching_prompt)
display(HTML(styled_table.to_html()))  